In [27]:
# Import libraries
import glob
import pandas as pd
from sklearn.cluster import KMeans

In [11]:
# Get CSV files list from a folder
path = 'Weather_Data'
csv_files = glob.glob(path + "/*.csv")

df_list = (pd.read_csv(file) for file in csv_files)

# Concatenate all DataFrames
weather_data   = pd.concat(df_list, ignore_index=True)

c:\Users\prati\AppData\Local\Programs\Python\Python38\lib\site-packages\pandas\core\reshape\concat.py:348: DtypeWarning: Columns (1,2,3,4,5,6,7,8,9,10,11,12) have mixed types.Specify dtype option on import or set low_memory=False.
  objs = list(objs)


In [12]:
weather_data.head()

,time,depth,latitude,longitude,Tdir,Tper,Thgt,sdir,sper,shgt,wdir,wper,whgt
0,UTC,m,degrees_north,degrees_east,degrees,second,meters,degrees,seconds,meters,degrees,seconds,meters
1,2022-09-08T19:00:00Z,0,-77.5,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2022-09-08T19:00:00Z,0,-77.5,0.5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,2022-09-08T19:00:00Z,0,-77.5,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,2022-09-08T19:00:00Z,0,-77.5,1.5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [13]:
def missing_statistics(df):    
    statitics = pd.DataFrame(df.isnull().sum()).reset_index()
    statitics.columns=['COLUMN NAME',"MISSING VALUES"]
    statitics['TOTAL ROWS'] = df.shape[0]
    statitics['% MISSING'] = round((statitics['MISSING VALUES']/statitics['TOTAL ROWS'])*100,2)
    return statitics

missing_statistics(weather_data)

,COLUMN NAME,MISSING VALUES,TOTAL ROWS,% MISSING
0,time,0,12020660,0.00
1,depth,0,12020660,0.00
2,latitude,0,12020660,0.00
3,longitude,0,12020660,0.00
4,Tdir,4622664,12020660,38.46
5,Tper,4621338,12020660,38.44
6,Thgt,4621338,12020660,38.44
7,sdir,5207365,12020660,43.32
8,sper,5207365,12020660,43.32
9,shgt,5207365,12020660,43.32


In [14]:
#Dropping rows with na
weather_data.dropna(inplace=True, axis=0)
weather_data.reset_index(drop=True, inplace=True)

In [16]:
weather_data.head()

,time,depth,latitude,longitude,Tdir,Tper,Thgt,sdir,sper,shgt,wdir,wper,whgt
0,UTC,m,degrees_north,degrees_east,degrees,second,meters,degrees,seconds,meters,degrees,seconds,meters
1,2022-09-08T19:00:00Z,0,-77,207.5,121,1.9193857,0.17,18,3.04,0.06,124,1.92,0.14999999
2,2022-09-08T19:00:00Z,0,-77,208,129,1.8832392,0.17,6,3.22,0.06,138,1.87,0.16
3,2022-09-08T19:00:00Z,0,-76.5,207.5,102,2.173913,0.14999999,21,2.8799999,0.06,110,2.1499999,0.14
4,2022-09-08T19:00:00Z,0,-76,211.5,66,2.5188916,0.12,38,2.87,0.08,109,2.1599998,0.089999996


In [23]:
# getting unique values of time ranges
allTimeRanges = weather_data["time"].iloc[1:].unique()

# removing all occurence of the word "UTC"
allTimeRanges = allTimeRanges[allTimeRanges != "UTC"]
allTimeRanges

array(['2022-09-08T19:00:00Z', '2022-09-08T20:00:00Z',
       '2022-09-08T21:00:00Z', '2022-09-08T22:00:00Z',
       '2022-09-08T23:00:00Z', '2022-09-09T05:00:00Z',
       '2022-09-09T06:00:00Z', '2022-09-09T07:00:00Z',
       '2022-09-09T08:00:00Z', '2022-09-09T09:00:00Z',
       '2022-09-08T00:00:00Z', '2022-09-08T01:00:00Z',
       '2022-09-08T02:00:00Z', '2022-09-08T03:00:00Z',
       '2022-09-08T04:00:00Z', '2022-09-08T05:00:00Z',
       '2022-09-08T06:00:00Z', '2022-09-08T07:00:00Z',
       '2022-09-08T08:00:00Z', '2022-09-08T09:00:00Z',
       '2022-09-08T10:00:00Z', '2022-09-08T11:00:00Z',
       '2022-09-08T12:00:00Z', '2022-09-08T13:00:00Z',
       '2022-09-08T14:00:00Z', '2022-09-08T15:00:00Z',
       '2022-09-08T16:00:00Z', '2022-09-08T17:00:00Z',
       '2022-09-08T18:00:00Z', '2022-09-09T00:00:00Z',
       '2022-09-09T01:00:00Z', '2022-09-09T02:00:00Z',
       '2022-09-09T03:00:00Z', '2022-09-09T04:00:00Z',
       '2022-09-09T10:00:00Z', '2022-09-09T11:00:00Z',
       '20

In [28]:
# working on only 1 time range for now
x_subset = weather_data.loc[weather_data["time"] == allTimeRanges[0]]
x_subset.shape

(140880, 13)

In [ ]:
# kmeans on this x_subset data
kmeans = KMeans(n_clusters=5)
x_subset.columns